In [ ]:
import numpy as np
import pandas as pd
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.0 MB/s eta 0:00:00


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv('../input/allflowmeterhikari2021v14/ALLFLOWMETER_HIKARI2022.csv')
df = df.drop(['Unnamed: 0','flow_duration', 'uid', 'originh', 'originp', 'responh', 'responp'], axis=1)
df.sample(10)

,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,flow_FIN_flag_count,flow_SYN_flag_count,flow_RST_flag_count,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_URG_flag_count,bwd_URG_flag_count,flow_CWR_flag_count,flow_ECE_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.min,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,attack_category,Label
192098,1,0,0,0,0.000000,0.000000,0.000000,0.000000,40,40,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,65535,0,65535,0,Benign,0
137042,2,1,0,0,12576.623688,6288.311844,18864.935532,0.500000,60,20,40,32,32,32,0,1,1,0,0,1,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,159.025192,1.590252e+02,1.590252e+02,159.025192,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,50.067902,1.089573e+02,1.590252e+02,79.512596,41.641086,0.000000,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.590252e+02,1.590252e+02,1.590252e+02,1.590252e+02,0.0,0.0,0.0,0.0,0.0,0.0,64240,64,0,64,Benign,0
152024,6,5,0,0,622.731466,518.942888,1141.674354,0.833333,200,32,40,168,32,40,2,2,0,0,0,10,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,220.775604,7.406235e+03,9.696960e+03,1939.392090,3064.626307,0.000000,7.700920e+03,9.343863e+03,2335.965633,3650.032209,0.000000,7.406235e+03,9.701014e+03,970.101357,2275.927076,0.000000,6.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,9.701014e+03,9.701014e+03,9.701014e+03,9.701014e+03,0.0,0.0,0.0,0.0,0.0,0.0,64240,65160,63,64,Benign,0
166848,12,12,3,6,4.388299,4.388299,8.776597,1.000000,392,32,40,392,32,40,2,2,0,3,6,23,0,0,0,0,0.0,517.0,889.0,74.083333,165.251576,0.0,2816.0,4781.0,398.416667,834.517221,0.0,2816.0,5670.0,236.250000,611.205171,0.000000,1.165854e+06,2.786741e+06,253340.092572,390661.594448,9.059906,1.067177e+06,2.734496e+06,248590.534384,357519.526882,0.000000,1.067177e+06,2.786741e+06,121162.652969,262074.080364,2073.471095,6.0,6.0,444.5,2390.5,0.0,0.0,0.0,0.0,0.0,0.000000,2.786741e+06,2.786741e+06,2.786741e+06,2.786741e+06,0.0,0.0,0.0,0.0,0.0,0.0,65535,65160,387,64,Benign,0
25349,85,65,14,59,39.752987,30.399343,70.152330,0.764706,2752,20,44,2088,32,40,2,2,2,14,23,147,0,0,0,0,0.0,357.0,1971.0,23.188235,68.380848,0.0,2896.0,118592.0,1824.492308,1284.399144,0.0,2896.0,120563.0,803.753333,1230.172564,5.960464,2.476690e+05,2.240727e+06,26675.323645,53958.295385,10.013580,3.509719e+05,2.138085e+06,33407.576382,75485.600883,5.960464,2.476690e+05,2.240727e+06,15038.437491,40810.884719,56385.169277,85.0,65.0,1971.0,118592.0,0.0,33000.0,0.0,13.0,0.0,486857.166977,2.240727e+06,2.240727e+06,2.240727e+06,2.240727e+06,0.0,0.0,0.0,0.0,0.0,0.0,642

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fwd_pkts_tot,228253.0,7.544996e+00,2.063005e+01,0.0,1.000000,6.000000,1.200000e+01,3.214000e+03
bwd_pkts_tot,228253.0,6.224299e+00,1.622458e+01,0.0,1.000000,5.000000,1.100000e+01,2.357000e+03
fwd_data_pkts_tot,228253.0,1.500913e+00,1.470135e+01,0.0,0.000000,0.000000,3.000000e+00,2.514000e+03
bwd_data_pkts_tot,228253.0,2.698269e+00,1.030832e+01,0.0,0.000000,0.000000,6.000000e+00,1.743000e+03
fwd_pkts_per_sec,228253.0,4.249762e+03,5.337921e+04,0.0,1.434792,64.759477,1.748720e+03,3.355443e+06
bwd_pkts_per_sec,228253.0,2.656370e+03,6.547922e+03,0.0,0.563013,58.353541,1.305707e+03,5.242880e+05
flow_pkts_per_sec,228253.0,6.906132e+03,5.448091e+04,0.0,1.962682,123.733911,3.083594e+03,3.355443e+06
down_up_ratio,228253.0,6.869020e-01,4.109304e-01,0.0,0.666667,0.833333,9.285710e-01,1.655556e+01
fwd_header_size_tot,228253.0,2.447868e+02,6.598387e+02,0.0,40.000000,200.000000,3.920000e+02,1.028560e+05
fwd_header_size_min,228253.0,3.104743e+01,8.824979e+00,0.0,32.000000,32.000000,3.200000e+01,4.400000e+01


In [ ]:
df.dtypes

fwd_pkts_tot                  int64
bwd_pkts_tot                  int64
fwd_data_pkts_tot             int64
bwd_data_pkts_tot             int64
fwd_pkts_per_sec            float64
bwd_pkts_per_sec            float64
flow_pkts_per_sec           float64
down_up_ratio               float64
fwd_header_size_tot           int64
fwd_header_size_min           int64
fwd_header_size_max           int64
bwd_header_size_tot           int64
bwd_header_size_min           int64
bwd_header_size_max           int64
flow_FIN_flag_count           int64
flow_SYN_flag_count           int64
flow_RST_flag_count           int64
fwd_PSH_flag_count            int64
bwd_PSH_flag_count            int64
flow_ACK_flag_count           int64
fwd_URG_flag_count            int64
bwd_URG_flag_count            int64
flow_CWR_flag_count           int64
flow_ECE_flag_count           int64
fwd_pkts_payload.min        float64
fwd_pkts_payload.max        float64
fwd_pkts_payload.tot        float64
fwd_pkts_payload.avg        

In [ ]:
df.nunique()

fwd_pkts_tot                   163
bwd_pkts_tot                   144
fwd_data_pkts_tot              100
bwd_data_pkts_tot              129
fwd_pkts_per_sec            125958
bwd_pkts_per_sec            116061
flow_pkts_per_sec           132039
down_up_ratio                  750
fwd_header_size_tot            717
fwd_header_size_min              9
fwd_header_size_max             11
bwd_header_size_tot            384
bwd_header_size_min              8
bwd_header_size_max             10
flow_FIN_flag_count             37
flow_SYN_flag_count              8
flow_RST_flag_count             11
fwd_PSH_flag_count              77
bwd_PSH_flag_count             114
flow_ACK_flag_count            220
fwd_URG_flag_count               1
bwd_URG_flag_count               1
flow_CWR_flag_count              3
flow_ECE_flag_count              2
fwd_pkts_payload.min           108
fwd_pkts_payload.max           356
fwd_pkts_payload.tot          1025
fwd_pkts_payload.avg          2426
fwd_pkts_payload.std

In [ ]:
df.dtypes

fwd_pkts_tot                  int64
bwd_pkts_tot                  int64
fwd_data_pkts_tot             int64
bwd_data_pkts_tot             int64
fwd_pkts_per_sec            float64
bwd_pkts_per_sec            float64
flow_pkts_per_sec           float64
down_up_ratio               float64
fwd_header_size_tot           int64
fwd_header_size_min           int64
fwd_header_size_max           int64
bwd_header_size_tot           int64
bwd_header_size_min           int64
bwd_header_size_max           int64
flow_FIN_flag_count           int64
flow_SYN_flag_count           int64
flow_RST_flag_count           int64
fwd_PSH_flag_count            int64
bwd_PSH_flag_count            int64
flow_ACK_flag_count           int64
fwd_URG_flag_count            int64
bwd_URG_flag_count            int64
flow_CWR_flag_count           int64
flow_ECE_flag_count           int64
fwd_pkts_payload.min        float64
fwd_pkts_payload.max        float64
fwd_pkts_payload.tot        float64
fwd_pkts_payload.avg        

In [ ]:
y = df['Label']
X = df.drop(['Label', 'attack_category'], axis=1)

del df

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=y)

In [ ]:
X_train = np.asarray(X_train)
X_val = np.asarray(X_val)
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)

In [ ]:
import torch
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier

unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.AdamW,
    optimizer_params=dict(lr=1e-2),
    mask_type='sparsemax')

unsupervised_model.fit(
    X_train=X_train,
    eval_set=[X_val],
    pretraining_ratio=0.8,
    max_epochs=10)

tabnet_clf = TabNetClassifier(
    optimizer_fn=torch.optim.AdamW,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"gamma":0.96},
    scheduler_fn=torch.optim.lr_scheduler.ExponentialLR,
    mask_type='sparsemax')

tabnet_clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    from_unsupervised=unsupervised_model,
    max_epochs=10)

/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.37841 | val_0_unsup_loss_numpy: 85.0485610961914|  0:00:12s
epoch 1  | loss: 1.08188 | val_0_unsup_loss_numpy: 11.584400177001953|  0:00:23s
epoch 2  | loss: 1.03501 | val_0_unsup_loss_numpy: 31.530729293823242|  0:00:33s
epoch 3  | loss: 0.9752  | val_0_unsup_loss_numpy: 15.49563980102539|  0:00:44s
epoch 4  | loss: 0.8902  | val_0_unsup_loss_numpy: 17.251419067382812|  0:00:55s
epoch 5  | loss: 0.94375 | val_0_unsup_loss_numpy: 296.7908020019531|  0:01:05s
epoch 6  | loss: 0.90402 | val_0_unsup_loss_numpy: 28.000699996948242|  0:01:16s
epoch 7  | loss: 0.88194 | val_0_unsup_loss_numpy: 40.7889404296875|  0:01:27s
epoch 8  | loss: 0.87346 | val_0_unsup_loss_numpy: 79.4503173828125|  0:01:37s
epoch 9  | loss: 0.8558  | val_0_unsup_loss_numpy: 91.2077865600586|  0:01:48s
Stop training because you reached max_epochs = 10 with best_epoch = 1 and best_val_0_unsup_loss_numpy = 11.584400177001953


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:231: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")


epoch 0  | loss: 0.08969 | train_auc: 0.99229 | valid_auc: 0.99281 |  0:00:11s
epoch 1  | loss: 0.05652 | train_auc: 0.99322 | valid_auc: 0.99279 |  0:00:24s
epoch 2  | loss: 0.04839 | train_auc: 0.99155 | valid_auc: 0.9921  |  0:00:36s
epoch 3  | loss: 0.04246 | train_auc: 0.99525 | valid_auc: 0.99525 |  0:00:48s
epoch 4  | loss: 0.03989 | train_auc: 0.99311 | valid_auc: 0.99323 |  0:01:01s
epoch 5  | loss: 0.03696 | train_auc: 0.99529 | valid_auc: 0.99493 |  0:01:13s
epoch 6  | loss: 0.03466 | train_auc: 0.9956  | valid_auc: 0.99548 |  0:01:26s
epoch 7  | loss: 0.03378 | train_auc: 0.99561 | valid_auc: 0.99543 |  0:01:38s
epoch 8  | loss: 0.03403 | train_auc: 0.99591 | valid_auc: 0.99567 |  0:01:50s
epoch 9  | loss: 0.03274 | train_auc: 0.99568 | valid_auc: 0.99583 |  0:02:03s
Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_valid_auc = 0.99583


/opt/conda/lib/python3.7/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from sklearn.metrics import roc_auc_score, classification_report, precision_recall_curve

val_preds = tabnet_clf.predict_proba(X_val)[:, -1]

precision, recall, thresholds = precision_recall_curve(y_val, val_preds)
f1_scores = 2*recall*precision/(recall+precision)
best_threshold = thresholds[np.argmax(f1_scores)]
print(f'Best threshold\t\t: {best_threshold}')
print(f'AUC score\t\t: {roc_auc_score(y_val, val_preds)}\n')

val_preds[val_preds > best_threshold] = 1
val_preds[val_preds <= best_threshold] = 0

print(classification_report(y_val, val_preds))

Best threshold		: 0.11710494756698608
AUC score		: 0.9958261149622163

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     42981
           1       0.72      1.00      0.84      2670

    accuracy                           0.98     45651
   macro avg       0.86      0.99      0.91     45651
weighted avg       0.98      0.98      0.98     45651

